Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [2]:
pickle_file = '../../nlp_data_set/notmnist/notMNIST_good.pickle'

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (192339, 28, 28) (192339,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (192339, 784) (192339, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
lr = 0.05
epochs = 100
batch_size =500
hidden_nodes = 1024
num_steps = 4001

- logistic

In [7]:
graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    print(train_X)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b1 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(train_X, W1), b1)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(train_X, W1), b1))

    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_X, W1), b1))

    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_X, W1), b1))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_dataset), size=batch_size, replace=False)
        batch_data = np.array([train_dataset[i] for i in rand_index])
        batch_labels = np.array([train_labels[i] for i in rand_index])
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Tensor("Placeholder:0", shape=(500, 784), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Initialized
Minibatch loss at step 0: 21.418232
Minibatch accuracy: 5.4%
Validation accuracy: 7.4%
Minibatch loss at step 500: 3.484394
Minibatch accuracy: 66.0%
Validation accuracy: 66.6%
Minibatch loss at step 1000: 2.427581
Minibatch accuracy: 69.4%
Validation accuracy: 71.2%
Minibatch loss at step 1500: 2.312521
Minibatch accuracy: 68.8%
Validation accuracy: 72.9%
Minibatch loss at step 2000: 1.979519
Minibatch accuracy: 76.0%
Validation accuracy: 73.5%
Minibatch loss at step 2500: 2.117676
Minibatch accuracy: 71.2%
Validation accuracy: 74.1%
Minibatch loss at step 3000: 2.024105
Minibatch accuracy: 71.6%
Validation accuracy: 74.5%
Minibatch loss at step 3500: 1.801237
Minibatch accuracy: 74.0%
Validation accuracy: 75.0%
Minibatch loss at

- neural network

In [8]:
graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    print(train_X)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    print(layer_1)

    # --------Create second layer (10 hidden nodes)--------
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(layer_1, W2), b2)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    pre_train_y = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(pre_train_y, W2), b2))

    valid_y = tf.nn.relu(tf.add(tf.matmul(valid_X, W1), b1))
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_y, W2), b2))

    test_y = tf.nn.relu(tf.add(tf.matmul(test_X, W1), b1))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_y, W2), b2))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_dataset), size=batch_size, replace=False)
        batch_data = np.array([train_dataset[i] for i in rand_index])
        batch_labels = np.array([train_labels[i] for i in rand_index])
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Tensor("Placeholder:0", shape=(500, 784), dtype=float32)
Tensor("Relu:0", shape=(500, 1024), dtype=float32)
Initialized
Minibatch loss at step 0: 458.301849
Minibatch accuracy: 3.4%
Validation accuracy: 13.3%
Minibatch loss at step 500: 22.485310
Minibatch accuracy: 79.0%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 13.349266
Minibatch accuracy: 82.6%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 10.241511
Minibatch accuracy: 82.0%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 9.217144
Minibatch accuracy: 85.4%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 6.348873
Minibatch accuracy: 85.6%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 5.506753
Minibatch accuracy: 86.4%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 8.929010
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 4000: 6.466455
Minibatch accuracy: 84.6%
Validation accuracy: 83.9%
Test accuracy: 90.0%


- logistic + L2

In [9]:
graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    print(train_X)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b1 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(train_X, W1), b1)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits)) + tf.nn.l2_loss(train_X)

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(train_X, W1), b1))

    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_X, W1), b1))

    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_X, W1), b1))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_dataset), size=batch_size, replace=False)
        batch_data = np.array([train_dataset[i] for i in rand_index])
        batch_labels = np.array([train_labels[i] for i in rand_index])
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Tensor("Placeholder:0", shape=(500, 784), dtype=float32)
Initialized
Minibatch loss at step 0: 41538.011719
Minibatch accuracy: 8.4%
Validation accuracy: 8.0%
Minibatch loss at step 500: 41476.851562
Minibatch accuracy: 63.4%
Validation accuracy: 66.5%
Minibatch loss at step 1000: 41664.761719
Minibatch accuracy: 69.6%
Validation accuracy: 70.8%
Minibatch loss at step 1500: 41655.105469
Minibatch accuracy: 70.8%
Validation accuracy: 72.4%
Minibatch loss at step 2000: 41479.281250
Minibatch accuracy: 74.8%
Validation accuracy: 73.5%
Minibatch loss at step 2500: 41050.328125
Minibatch accuracy: 72.8%
Validation accuracy: 74.1%
Minibatch loss at step 3000: 41520.636719
Minibatch accuracy: 73.2%
Validation accuracy: 74.4%
Minibatch loss at step 3500: 41421.093750
Minibatch accuracy: 75.4%
Validation accuracy: 74.7%
Minibatch loss at step 4000: 41751.003906
Minibatch accuracy: 76.6%
Validation accuracy: 75.1%
Test accuracy: 82.2%


- neural network + L2

In [10]:
graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    print(train_X)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    print(layer_1)

    # --------Create second layer (10 hidden nodes)--------
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(layer_1, W2), b2)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))  + tf.nn.l2_loss(train_X)

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    pre_train_y = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(pre_train_y, W2), b2))

    valid_y = tf.nn.relu(tf.add(tf.matmul(valid_X, W1), b1))
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_y, W2), b2))

    test_y = tf.nn.relu(tf.add(tf.matmul(test_X, W1), b1))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_y, W2), b2))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_dataset), size=batch_size, replace=False)
        batch_data = np.array([train_dataset[i] for i in rand_index])
        batch_labels = np.array([train_labels[i] for i in rand_index])
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Tensor("Placeholder:0", shape=(500, 784), dtype=float32)
Tensor("Relu:0", shape=(500, 1024), dtype=float32)
Initialized
Minibatch loss at step 0: 41641.222656
Minibatch accuracy: 10.0%
Validation accuracy: 17.5%
Minibatch loss at step 500: 41628.589844
Minibatch accuracy: 78.6%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 41419.511719
Minibatch accuracy: 81.6%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 41164.906250
Minibatch accuracy: 80.6%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 41691.425781
Minibatch accuracy: 82.6%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 41631.378906
Minibatch accuracy: 81.8%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 41711.976562
Minibatch accuracy: 84.8%
Validation accuracy: 82.4%
Minibatch loss at step 3500: 41594.339844
Minibatch accuracy: 85.0%
Validation accuracy: 82.9%
Minibatch loss at step 4000: 41822.398438
Minibatch accuracy: 88.0%
Validation accuracy: 83.5%
Test accuracy: 89.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    print(train_X)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    print(layer_1)

    # --------Create second layer (10 hidden nodes)--------
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(layer_1, W2), b2)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    pre_train_y = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(pre_train_y, W2), b2))

    valid_y = tf.nn.relu(tf.add(tf.matmul(valid_X, W1), b1))
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_y, W2), b2))

    test_y = tf.nn.relu(tf.add(tf.matmul(test_X, W1), b1))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_y, W2), b2))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    train_candidates_x = []
    train_candidates_y = []
    for _ in range(10):
        rand_index = np.random.choice(len(train_dataset), size=batch_size, replace=False)
        data_x = np.array([train_dataset[i] for i in rand_index])
        labels_y = np.array([train_labels[i] for i in rand_index])
        train_candidates_x.append(data_x)
        train_candidates_y.append(labels_y)
        
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_candidates_x))
        batch_data = train_candidates_x[rand_index]
        batch_labels = train_candidates_y[rand_index]
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Tensor("Placeholder:0", shape=(500, 784), dtype=float32)
Tensor("Relu:0", shape=(500, 1024), dtype=float32)
Initialized
Minibatch loss at step 0: 289.822540
Minibatch accuracy: 15.2%
Validation accuracy: 25.1%
Minibatch loss at step 500: 0.356599
Minibatch accuracy: 97.4%
Validation accuracy: 74.2%
Minibatch loss at step 1000: 0.001074
Minibatch accuracy: 100.0%
Validation accuracy: 72.7%
Minibatch loss at step 1500: 0.000007
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 2000: 0.000016
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 2500: 0.000013
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 3000: 0.000010
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 3500: 0.000005
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 4000: 0.000018
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Test accuracy: 81.5%


- 在训练集上达到100%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [15]:
train_candidates_x = []
train_candidates_y = []
for _ in range(10):
    rand_index = np.random.choice(len(train_dataset), size=batch_size, replace=False)
    data_x = np.array([train_dataset[i] for i in rand_index])
    labels_y = np.array([train_labels[i] for i in rand_index])
    train_candidates_x.append(data_x)
    train_candidates_y.append(labels_y)

In [16]:
keep_prob = 1

graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    layer_1 = tf.nn.dropout(layer_1, keep_prob)

    # --------Create second layer (10 hidden nodes)--------
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(layer_1, W2), b2)
    logits = tf.nn.dropout(logits, keep_prob)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    pre_train_y = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(pre_train_y, W2), b2))

    valid_y = tf.nn.relu(tf.add(tf.matmul(valid_X, W1), b1))
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_y, W2), b2))

    test_y = tf.nn.relu(tf.add(tf.matmul(test_X, W1), b1))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_y, W2), b2))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_candidates_x))
        batch_data = train_candidates_x[rand_index]
        batch_labels = train_candidates_y[rand_index]
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 329.910889
Minibatch accuracy: 8.8%
Validation accuracy: 16.2%
Minibatch loss at step 500: 2.256983
Minibatch accuracy: 92.4%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 0.016398
Minibatch accuracy: 99.8%
Validation accuracy: 73.5%
Minibatch loss at step 1500: 0.000024
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Minibatch loss at step 2000: 0.000054
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Minibatch loss at step 2500: 0.000016
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Minibatch loss at step 3000: 0.009983
Minibatch accuracy: 99.8%
Validation accuracy: 73.7%
Minibatch loss at step 3500: 0.000013
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Minibatch loss at step 4000: 0.000019
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Test accuracy: 81.0%


In [17]:
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    layer_1 = tf.nn.dropout(layer_1, keep_prob)

    # --------Create second layer (10 hidden nodes)--------
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(layer_1, W2), b2)
    logits = tf.nn.dropout(logits, keep_prob)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    pre_train_y = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(pre_train_y, W2), b2))

    valid_y = tf.nn.relu(tf.add(tf.matmul(valid_X, W1), b1))
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_y, W2), b2))

    test_y = tf.nn.relu(tf.add(tf.matmul(test_X, W1), b1))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_y, W2), b2))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_candidates_x))
        batch_data = train_candidates_x[rand_index]
        batch_labels = train_candidates_y[rand_index]
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 766.628418
Minibatch accuracy: 9.0%
Validation accuracy: 20.8%
Minibatch loss at step 500: 31.851259
Minibatch accuracy: 88.6%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 21.726175
Minibatch accuracy: 97.0%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 11.055876
Minibatch accuracy: 99.2%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 5.968597
Minibatch accuracy: 98.8%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 4.931558
Minibatch accuracy: 99.0%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 7.255889
Minibatch accuracy: 98.8%
Validation accuracy: 83.1%
Minibatch loss at step 3500: 4.007806
Minibatch accuracy: 99.6%
Validation accuracy: 83.2%
Minibatch loss at step 4000: 3.499869
Minibatch accuracy: 99.8%
Validation accuracy: 83.1%
Test accuracy: 89.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [37]:
keep_prob = 0.5
hidden_layer_2 = 1500
lr = 0.01
batch_size =600
hidden_nodes = 500
num_steps = 2001

graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
#     layer_1 = tf.nn.dropout(layer_1, keep_prob)

    # --------Create second layer (300 hidden nodes)--------
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_layer_2]))
    b2 = tf.Variable(tf.zeros([hidden_layer_2]))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, W2), b2))
#     layer_2 = tf.nn.dropout(layer_2, keep_prob)
    
    # --------Create last layer (10 nodes)--------
    W3 = tf.Variable(tf.truncated_normal([hidden_layer_2, num_labels]))
    b3 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(layer_2, W3), b3)
#     logits = tf.nn.dropout(logits, keep_prob)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    pre_train_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    pre_train_2 = tf.nn.relu(tf.add(tf.matmul(pre_train_1, W2), b2))
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(pre_train_2, W3), b3))

    valid_y_1 = tf.nn.relu(tf.add(tf.matmul(valid_X, W1), b1))
    valid_y_2 = tf.nn.relu(tf.add(tf.matmul(valid_y_1, W2), b2))
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_y_2, W3), b3))

    test_y_1 = tf.nn.relu(tf.add(tf.matmul(test_X, W1), b1))
    test_y_2 = tf.nn.relu(tf.add(tf.matmul(test_y_1, W2), b2))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_y_2, W3), b3))


with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print(offset)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

Initialized
0
Minibatch loss at step 0: 6223.936523
Minibatch accuracy: 12.7%
Validation accuracy: 24.9%
108261
Minibatch loss at step 500: 114.137344
Minibatch accuracy: 83.7%
Validation accuracy: 79.8%
24783
Minibatch loss at step 1000: 72.170883
Minibatch accuracy: 84.3%
Validation accuracy: 82.3%
133044
Minibatch loss at step 1500: 56.974155
Minibatch accuracy: 85.0%
Validation accuracy: 82.7%
49566
Minibatch loss at step 2000: 37.122707
Minibatch accuracy: 88.3%
Validation accuracy: 83.1%
Test accuracy: 89.1%
